<a href="https://colab.research.google.com/github/NadaSOsman/MobilityPred-COVID-Restrictions/blob/master/Autoregressive_one_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
from os import listdir
from os.path import isfile, join
import gzip
import shutil

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('/root/data')
try:
	os.makedirs(local_download_path)
except: pass

file_list = drive.ListFile(
	{'q': "'1aLmDCh-VHHvZExBxHyesB4W2j9KAnRlt' in parents"}).GetList()

for f in file_list:
	# 3. Create & download by id.
	print('title: %s, id: %s' % (f['title'], f['id']))
	fname = os.path.join(local_download_path, f['title'])
	print('downloading to {}'.format(fname))
	f_ = drive.CreateFile({'id': f['id']})
	f_.GetContentFile(fname)
 

import numpy as np
import tensorflow as tf
import random as rn
import sys

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(1442)
rn.seed(12435)

from tensorflow.compat.v1.keras import backend as K

tf.random.set_seed(1234)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras import optimizers
import csv
import random
import gc

INPUT_WIN_SIZE = 10
OUTPUT_WIN_SIZE = 5
MOBILITY_CATEGORIES = 6
TRAINIG_DATA_SAMPLES = 99
TESTING_DATA_SAMPLES = 20
SAMPLE_SIZE = 164
BATCH_SIZE = 90


with open("/root/data/training-moblity.csv", 'r', encoding="utf8") as f:
  reader = csv.reader(f, delimiter=',')
  headers = next(reader)
  training_data = np.array(list(reader)).astype(int)
f.close()

with open("/root/data/testing-mobility.csv", 'r', encoding="utf8") as f:
  reader = csv.reader(f, delimiter=',')
  headers = next(reader)
  testing_data = np.array(list(reader)).astype(int)
f.close()

def build_model():
  inputs = []
  outputs = []
  for i in range(0, MOBILITY_CATEGORIES):
    input = Input(shape=(INPUT_WIN_SIZE,), dtype='int32')
    fc1 = Dense(32, activation='relu')(input)
    d1 = Dropout(0.2)(fc1)
    fc2 = Dense(16, activation='relu')(d1)
    d2 = Dropout(0.2)(fc2)
    fc3 = Dense(8, activation='relu')(d2)
    d3 = Dropout(0.2)(fc3)
    output = Dense(1, activation='linear')(input)
    
    inputs.append(input)
    outputs.append(output)

  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(learning_rate=0.001))

  print(model.summary())
  return model

global current_training_indeces
def generator(training):
  global current_training_indeces
  if (training == 1):
    start_batch = random.randint(0,TRAINIG_DATA_SAMPLES)
    indeces = [i*SAMPLE_SIZE for i in range(start_batch, TRAINIG_DATA_SAMPLES)]
    diff = BATCH_SIZE-len(indeces)
    for i in range (0, diff):
      indeces.append(i*SAMPLE_SIZE)
    current_training_indeces = indeces[:]
    data = np.transpose(training_data)
  elif (training == 2):
    indeces = []
    for i in range(0, TRAINIG_DATA_SAMPLES):
      if (i*SAMPLE_SIZE not in current_training_indeces):
        indeces.append(i*SAMPLE_SIZE)
    data = np.transpose(training_data)
  else:
    indeces = [i*i*SAMPLE_SIZE for i in range(0,TESTING_DATA_SAMPLES)]
    data = np.transpose(testing_data)

  time_series_input = [[] for i in range(0, MOBILITY_CATEGORIES)]
  time_series_output = [[] for i in range(0, MOBILITY_CATEGORIES)]

  for index in indeces:
    i = index
    while i < index+SAMPLE_SIZE-(INPUT_WIN_SIZE+OUTPUT_WIN_SIZE):
      for c in range(0, MOBILITY_CATEGORIES):
        time_series_input[c].append(np.reshape(data[c][i:i+INPUT_WIN_SIZE],(INPUT_WIN_SIZE,)))
        time_series_output[c].append(np.reshape(data[c][i+INPUT_WIN_SIZE:i+INPUT_WIN_SIZE+OUTPUT_WIN_SIZE],(OUTPUT_WIN_SIZE,)))
      i += 1

  for c in range(0, MOBILITY_CATEGORIES):
     time_series_input[c] = np.asarray(time_series_input[c])
     time_series_output[c] = np.asarray(time_series_output[c])

  return time_series_input, time_series_output

def validation_batch_generator():
	while True:
		gc.collect()
		time_series_input, time_series_output = generator(2)
		yield time_series_input, time_series_output

def training_batch_generator():
	while True:
		gc.collect()
		time_series_input, time_series_output = generator(1)
		yield time_series_input, time_series_output

def train_model(model):
  model.fit_generator(generator = training_batch_generator(), steps_per_epoch=1, epochs=150, verbose=1, validation_data=validation_batch_generator(), validation_steps=1)
  return model


model = build_model()
model = train_model(model)


title: Autoregressive-one-layer.ipynb, id: 1cSAZPkJ9X6NmDEZsQzeZ9egQHrUaAp9z
downloading to /root/data/Autoregressive-one-layer.ipynb
title: Model.ipynb, id: 1rbdqYWX7UVZaCEyk6kFNGhs0t5HDMPYT
downloading to /root/data/Model.ipynb
title: Control_Measures_Model_with_Duration.ipynb, id: 1mWKqG_cp9HE3PL6JC14jQny3kE6ScKqw
downloading to /root/data/Control_Measures_Model_with_Duration.ipynb
title: Control_Measures_Model_Without_Autoregressive.ipynb, id: 1eqLe-YDi_ATWwGiGQXt-Zjfkzq8BKNgm
downloading to /root/data/Control_Measures_Model_Without_Autoregressive.ipynb
title: Control_Measures_Model.ipynb, id: 1ZsIdI0GmpozFUY7YZ9DrVA4UKma13bKQ
downloading to /root/data/Control_Measures_Model.ipynb
title: Autoregressive.ipynb, id: 1yFzL26YHZxbkLbMTPPmNzjaVd3EjzdiF
downloading to /root/data/Autoregressive.ipynb
title: testing-control-duration.csv, id: 1sndwIkTwLAwtyCatC2kMPr6fYfVFFUKU
downloading to /root/data/testing-control-duration.csv
title: training-control-duration.csv, id: 1rajpMsOBPG4KbJurE2t